# **Data Analysis of Cricket Match Dataset: Test Nations (1877–2025):**

In [3]:
import pandas as pd
import numpy as np
import sys
import warnings
from datetime import datetime
warnings.filterwarnings("ignore") # Ignore warnings
import re

from data_analysis_pipeline import data_engineering

data= data_engineering()
data.shape

(7002, 19)

In [2]:
data.head()

,team_1,team_2,winner,margin,ground,format,test_score,odi_score,t20i_score,start_date,year_month,match_duration,year,month,ground_country,is_neutral_ground,won_by_wickets,won_by_runs,won_by_inns
2,India,Pakistan,India,6 wickets,Delhi,Test,1849,0,0,2007-11-22,2007-Nov,5,2007,Nov,India,False,6.0,0.0,False
3,Pakistan,India,Pakistan,341 runs,Karachi,Test,1783,0,0,2006-01-29,2006-Jan,4,2006,Jan,Pakistan,False,0.0,341.0,False
6,India,Pakistan,Pakistan,168 runs,Bengaluru,Test,1743,0,0,2005-03-24,2005-Mar,5,2005,Mar,India,False,0.0,168.0,False
7,India,Pakistan,India,195 runs,Eden Gardens,Test,1741,0,0,2005-03-16,2005-Mar,5,2005,Mar,India,False,0.0,195.0,False
9,Pakistan,India,India,inns & 131 runs,Rawalpindi,Test,1697,0,0,2004-04-13,2004-Apr,4,2004,Apr,Pakistan,False,0.0,131.0,False


In [5]:
data.columns

Index(['team_1', 'team_2', 'winner', 'margin', 'ground', 'format',
       'test_score', 'odi_score', 't20i_score', 'start_date', 'year_month',
       'match_duration', 'year', 'month', 'ground_country',
       'is_neutral_ground', 'won_by_wickets', 'won_by_runs', 'won_by_inns'],
      dtype='object')